In [ ]:
# @title Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Text Embeddings on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/text_embeddings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fgenerative_ai%2Ftext_embeddings.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/text_embeddings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/text_embedding.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook will walk you through the process of getting text embeddings given a text-embedding model such as **text-embedding-005** and **text-multilingual-embedding-002**. See also [getting text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#api_changes_to_models_released_in_or_after_august_2023).

### Objective

In this tutorial, you learn how to get a text embedding given a text-embedding model and a text.

This tutorial uses the following Google Cloud ML services and resources:

* Vertex AI

The steps include:

* Installing and initializing Vertex AI SDK.
* Authenticating your Google Cloud account.
* Getting an embedding given a model and a text.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get Started

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/projectselector2/). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,documentai.googleapis.com).

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Installation

This tutorial requires you to install the latest `google-cloud-aiplatform` package.

In [ ]:
!pip3 install --quiet --upgrade google-cloud-aiplatform

#### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Initialize Vertex AI SDK

Import and Initialize the SDK for your project and region.

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See also [locating the project ID](https://support.google.com/googleapi/answer/7014113).

See also [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
# @title (Required) Set PROJECT_ID and REGION
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}
if not PROJECT_ID.strip():
    raise ValueError("'PROJECT_ID' is required.")
if not REGION.strip():
    raise ValueError("'REGION' is required.")
!gcloud config set project {PROJECT_ID}

import vertexai
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

vertexai.init(project=PROJECT_ID, location=REGION)

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Colab:**

In [ ]:
# @title (Required on Colab) `authenticate_user()`
import builtins
import os
import sys

running_in_colab = "google.colab" in sys.modules and hasattr(builtins, "get_ipython")
if running_in_colab and not os.getenv("IS_TESTING"):
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

**2. Vertex AI Workbench**

Make sure that [the Compute Engine default service account](https://cloud.google.com/compute/docs/access/service-accounts#default_service_account) running a Vertex AI workbench instance has the permission iam.serviceAccounts.actAs (most likely through [roles/iam.serviceAccountUser](https://cloud.google.com/iam/docs/understanding-roles#iam.serviceAccountUser)) at [the IAM & Admin page of the Cloud Console](https://console.cloud.google.com/iam-admin). This permission allows a workbench instance to act as the service account when interacting with other Google Cloud services.

**3. Local JupyterLab instance, uncomment and run:**

In [ ]:
# !gcloud auth login

### Get Text Embeddings

1.   Specify the model. The latest models are
     * **text-embedding-005** for English.
     * **text-multilingual-embedding-002** for supported, international languages.
     
     See the [supported text languages](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api#supported_text_languages) and the [supported language models](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#supported-models).
2.   Specify the task, text, and optional title (valid only for task **RETRIEVAL_DOCUMENT**). The valid task types are:
     * **RETRIEVAL_QUERY**
     * **RETRIEVAL_DOCUMENT**
     * **SEMANTIC_SIMILARITY**
     * **CLASSIFICATION**
     * **CLUSTERING**
     * **QUESTION_ANSWERING** (valid only for the latest models)
     * **FACT_VERIFICATION** (valid only for the latest models)
     * **CODE_RETRIEVAL_QUERY** (valid only for the latest models)
3. Optionally specify the output dimensionality (valid only for the latest models).

In [ ]:
# @title { run: "auto" }
MODEL = "text-embedding-005"  # @param ["text-embedding-005", "text-embedding-004", "text-multilingual-embedding-002"]
TASK = "CODE_RETRIEVAL_QUERY"  # @param ["RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT", "SEMANTIC_SIMILARITY", "CLASSIFICATION", "CLUSTERING", "QUESTION_ANSWERING", "FACT_VERIFICATION", "CODE_RETRIEVAL_QUERY"]
TEXT = "Banana Muffin?"  # @param {type:"string"}
TITLE = ""  # @param {type:"string"}
OUTPUT_DIMENSIONALITY = 256  # @param [1, 768, "None"] {type:"raw", allow-input:true}

if not MODEL:
    raise ValueError("MODEL must be specified.")
if not TEXT:
    raise ValueError("TEXT must be specified.")
if TITLE and TASK != "RETRIEVAL_DOCUMENT":
    raise ValueError("TITLE can only be specified for TASK 'RETRIEVAL_DOCUMENT'")
if OUTPUT_DIMENSIONALITY is not None and MODEL not in [
    "text-embedding-005",
    "text-embedding-004",
    "text-multilingual-embedding-002",
]:
    raise ValueError(f"OUTPUT_DIMENTIONALITY cannot be specified for model '{MODEL}'.")
if TASK in ["QUESTION_ANSWERING", "FACT_VERIFICATION"] and MODEL not in [
    "text-embedding-005",
    "text-embedding-004",
    "text-multilingual-embedding-002",
]:
    raise ValueError(f"TASK '{TASK}' is not valid for model '{MODEL}'.")
if TASK in ["CODE_RETRIEVAL_QUERY"] and MODEL not in [
    "text-embedding-005",
]:
    raise ValueError(f"TASK '{TASK}' is not valid for model '{MODEL}'.")

In [ ]:
def embed_text(
    model_name: str,
    task_type: str,
    text: str,
    title: str = "",
    output_dimensionality=None,
) -> list:
    """Generates a text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    text_embedding_input = TextEmbeddingInput(
        task_type=task_type, title=title, text=text
    )
    kwargs = (
        dict(output_dimensionality=output_dimensionality)
        if output_dimensionality
        else {}
    )
    embeddings = model.get_embeddings([text_embedding_input], **kwargs)
    return embeddings[0].values


# Get a text embedding for a downstream task.
embedding = embed_text(
    model_name=MODEL,
    task_type=TASK,
    text=TEXT,
    title=TITLE,
    output_dimensionality=OUTPUT_DIMENSIONALITY,
)
print(len(embedding))  # Expected value: {OUTPUT_DIMENSIONALITY}.

### Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.